# Chargement des fichiers sources

### Conversion des images du pdf en text

In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter
from PyPDF2.generic import (
    DictionaryObject,
    NumberObject,
    NameObject,
    ArrayObject,
    StreamObject,
    TextStringObject,
    IndirectObject
)
sources_path = os.path.join(os.getcwd(), "sources")
INPUT_PDF = "output_uncompressed.pdf"
OUTPUT_PDF = sources_path + "/modified.pdf"

reader = PdfReader(sources_path + '/' + INPUT_PDF)
writer = PdfWriter()

# ID de la police à réutiliser pour écrire le texte "[crane]"
FONT_REF = IndirectObject(120, 0, reader)  # correspond à /C0_0 120 0 R

# Crée un Form XObject qui affiche le texte [crane]
form_stream = StreamObject()
form_stream._data = b"BT /F1 12 Tf 0 0 Td ([crane]) Tj ET"
form_stream.update({
    NameObject("/Type"): NameObject("/XObject"),
    NameObject("/Subtype"): NameObject("/Form"),
    NameObject("/BBox"): ArrayObject([NumberObject(0), NumberObject(0), NumberObject(100), NumberObject(20)]),
    NameObject("/Resources"): DictionaryObject({
        NameObject("/Font"): DictionaryObject({
            NameObject("/F1"): FONT_REF
        })
    }),
})

# Ajoute le nouvel objet au PDF
form_obj_number = writer._objects.__len__()
writer._add_object(form_stream)
replacement_ref = IndirectObject(form_obj_number, 0, writer)

for page in reader.pages:
    resources = page.get("/Resources")
    xobjects = resources.get("/XObject")

    if not xobjects:
        writer.add_page(page)
        continue

    modified = False

    # Remplace l'entrée /Im2 si elle pointe vers 1326 0 R
    new_xobject_dict = DictionaryObject()
    for key, obj in xobjects.items():
        if key == NameObject("/Im2") and isinstance(obj, IndirectObject):
            if obj.idnum == 1326:
                new_xobject_dict[NameObject("/Im2")] = replacement_ref
                modified = True
                continue
        new_xobject_dict[key] = obj

    if modified:
        new_resources = DictionaryObject(resources)
        new_resources[NameObject("/XObject")] = new_xobject_dict
        page[NameObject("/Resources")] = new_resources

    writer.add_page(page)

with open(OUTPUT_PDF, "wb") as f:
    writer.write(f)

print("✅ Fichier modifié enregistré dans :", OUTPUT_PDF)


🛠️ Remplacement sur une page...


AttributeError: 'bytes' object has no attribute 'indirect_reference' and no __dict__ for setting new attributes

In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter
from PyPDF2.generic import (
    DictionaryObject,
    NumberObject,
    NameObject,
    ArrayObject,
    StreamObject,
    TextStringObject,
    IndirectObject
)
sources_path = os.path.join(os.getcwd(), "sources")
INPUT_PDF = "output_uncompressed.pdf"
OUTPUT_PDF = sources_path + "/modified.pdf"

reader = PdfReader(sources_path + '/' + INPUT_PDF)
writer = PdfWriter()

# ID de la police à réutiliser pour écrire le texte "[crane]"
FONT_REF = IndirectObject(120, 0, reader)  # correspond à /C0_0 120 0 R

for page in reader.pages:
    content_object = page.get_contents()
    if not content_object:
        writer.add_page(page)
        continue

    content_data = b""
    if isinstance(content_object, list):
        for obj in content_object:
            content_data += obj.get_data()
    else:
        content_data = content_object.get_data()

     # Remplace l'image /Im2 Do par un affichage de texte
    # Instruction PDF pour afficher du texte : BT /F1 12 Tf 0 0 Td ([crane]) Tj ET
    # Attention, cela suppose qu'une police /F1 existe dans les /Resources

    if b"/Im2 Do" in content_data:
        print("🔁 Remplacement sur une page")

        # Assure que la font /F1 existe dans les ressources
        resources = page["/Resources"]
        if "/Font" not in resources:
            resources[NameObject("/Font")] = DictionaryObject()

        font_dict = resources["/Font"]
        if "/F1" not in font_dict:
            # Utilise une police existante dans la page, ici on utilise C0_0
            if "/C0_0" in font_dict:
                font_dict[NameObject("/F1")] = font_dict["/C0_0"]
            else:
                # On abandonne si aucune font n'existe
                print("⚠️ Aucune font trouvée pour insérer le texte.")
                writer.add_page(page)
                continue
        
        # Construction d'une instruction texte en PDF
        # Tu peux ajuster la position (Td) et la taille (Tf)
        replacement = b"BT /F1 12 Tf 10 10 Td ([crane]) Tj ET"

        # Remplace toutes les occurrences
        modified_content = content_data.replace(b"/Im2 Do", replacement)

        # Injecte le contenu modifié dans un nouveau stream
        page.__setitem__(NameObject("/Contents"), writer._add_stream(modified_content))

    writer.add_page(page)

with open(OUTPUT_PDF, "wb") as f:
    writer.write(f)

print("✅ Fichier modifié enregistré dans :", OUTPUT_PDF)


### transformation PDF -> TXT avec PyPDF

In [17]:
import os
from PyPDF2 import PdfReader

# Import Files 
def file_loader(folder):
    files = []
    for i, file_name in enumerate(os.listdir(folder)):
        file_path = os.path.join(folder, file_name)
        if os.path.isfile(file_path):
            extension = os.path.splitext(file_name)[1].lower()
            with open(file_path, 'r', encoding='latin-1') as f:
                files.append({
                    'name': file_name,
                    'path': file_path,
                    'extension': extension,
                    'content': f.read(),
                })
            print(f"Chargé : {files[i]['name']} ({len(files[i]['content'])} caractères) {files[i]['extension']})")
    return files

# Convert PDF to TXT
def pdf_to_txt(file, output_folder_path):
    """
    Convert a PDF file (already loaded as a dict from file_loader) to TXT and save it in output_folder.
    Returns the path to the TXT file.
    """
    txt_name = os.path.splitext(file['name'])[0] + '.txt'
    txt_path = os.path.join(output_folder_path, txt_name)
    if file['extension'] == '.pdf':
        if not os.path.isfile(txt_path):
            print(f"Conversion du PDF {file['name']} en TXT...")
            with open(file['path'], 'rb') as f:
                reader = PdfReader(f)
                text = ""
                for page in reader.pages:
                    text += page.extract_text() or ""
            with open(txt_path, 'w', encoding='utf-8') as f_txt:
                f_txt.write(text)
            print(f"Fichier TXT créé : {os.path.splitext(file['name'])[0]}.txt")
        else:
            print(f"Le fichier TXT {txt_name} existe déjà, pas de conversion nécessaire.")
        return txt_path
    print(f"Le fichier {file['name']} n'est pas un PDF, pas de conversion effectuée.")
    return None

sources_path = os.path.join(os.getcwd(), "sources")
transformed_sources_path = os.path.join(os.getcwd(), "transformed_sources")
files = file_loader(sources_path)
for file in files:
    if(file['extension'] == '.pdf'):
        pdf_to_txt(file, transformed_sources_path)

Chargé : Warhammer 4 - Livre de base.pdf (67641992 caractères) .pdf)
Le fichier TXT Warhammer 4 - Livre de base.txt existe déjà, pas de conversion nécessaire.


### transformation PDF -> TXT avec pdfplumber

In [ ]:
import os
import pdfplumber

# Import Files 
def file_loader(folder):
    files = []
    for i, file_name in enumerate(os.listdir(folder)):
        file_path = os.path.join(folder, file_name)
        if os.path.isfile(file_path):
            extension = os.path.splitext(file_name)[1].lower()
            with open(file_path, 'r', encoding='latin-1', errors='ignore') as f:
                files.append({
                    'name': file_name,
                    'path': file_path,
                    'extension': extension,
                    'content': f.read(),
                })
            print(f"Chargé : {files[i]['name']} ({len(files[i]['content'])} caractères) {files[i]['extension']})")
    return files

# Convert PDF to TXT
def pdf_to_txt(file, output_folder_path):
    """
    Convert a PDF file (already loaded as a dict from file_loader) to TXT and save it in output_folder.
    Returns the path to the TXT file.
    """
    txt_name = os.path.splitext(file['name'])[0] + '.txt'
    txt_path = os.path.join(output_folder_path, txt_name)
    
    if file['extension'] == '.pdf':
        if not os.path.isfile(txt_path):
            print(f"Conversion du PDF {file['name']} en TXT...")
            text = ""
            with pdfplumber.open(file['path']) as pdf:
                for page in pdf.pages:
                    text += page.extract_text() or ""
            with open(txt_path, 'w', encoding='utf-8') as f_txt:
                f_txt.write(text)
            print(f"Fichier TXT créé : {txt_name}")
        else:
            print(f"Le fichier TXT {txt_name} existe déjà, pas de conversion nécessaire.")
        return txt_path
    
    print(f"Le fichier {file['name']} n'est pas un PDF, pas de conversion effectuée.")
    return None

# Répertoires
sources_path = os.path.join(os.getcwd(), "sources")
transformed_sources_path = os.path.join(os.getcwd(), "transformed_sources2")

# Traitement
files = file_loader(sources_path)
for file in files:
    if file['extension'] == '.pdf':
        pdf_to_txt(file, transformed_sources_path)

# Découpage des fichiers txt en chunks

In [ ]:
import os
from langchain.text_splitter import TokenTextSplitter
import tiktoken
import json

# Paramètres des chunks
SOURCES_DIR = "transformed_sources"                             # Dossier contenant les fichiers texte à découper
CHUNK_SIZE = 1000                                                # Nombre de tokens par chunk
CHUNK_OVERLAP = 200                                              # Nombre de tokens de chevauchement
OUTPUT_DIR = f"chunks/CS_{CHUNK_SIZE}_CO_{CHUNK_OVERLAP}"       # Dossier de sortie pour les chunks

encoding = tiktoken.get_encoding("cl100k_base")     # Utiliser l'encodeur de tokens de OpenAI (compatible Mistral)

# Initialiser le découpeur de texte
splitter = TokenTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    encoding_name="cl100k_base",
)

def read_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def write_json(path, config):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(config, f, indent=4, ensure_ascii=False)

def chunk_files(new_files, old_files):
    i = 0
    for i, new_file in enumerate(new_files, start=1):
        print(f"Découpage de {new_file} en chunk de {CHUNK_SIZE} tokens...")
        chunk_file(os.path.join(SOURCES_DIR, new_file))

    print(f"{i} fichiers ont été découpés en chunks (dans '{OUTPUT_DIR}/').")
    # Enregistre les paramètres de découpage et les fichiers traités dans un fichier JSON
    new_param = {
        "CHUNK_SIZE": CHUNK_SIZE,
        "CHUNK_OVERLAP": CHUNK_OVERLAP,
        "SOURCE_DIR": SOURCES_DIR,
        "files": old_files + new_files,
    }
    write_json(os.path.join(OUTPUT_DIR, "param.json"), new_param)

# Découpe un fichier texte en chunks et les enregistre dans le dossier de sortie
def chunk_file(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        text = f.read()
    chunks = splitter.split_text(text)

    filename = os.path.splitext(os.path.basename(filepath))[0]
    for i, chunk in enumerate(chunks):
        with open(os.path.join(OUTPUT_DIR, f"{filename}_chunk_{i}.txt"), "w", encoding="utf-8") as out:
            out.write(chunk)


# vérifie et compare les fichiers ayant déjà été découpés avec les nouveaux fichiers
# afin de ne pas créer de doublons de chunks
def chunk_cleaner(files):
    param = read_json(os.path.join(OUTPUT_DIR, "param.json"))

    p_chunk_size = param.get("CHUNK_SIZE")
    p_chunk_overlap = param.get("CHUNK_OVERLAP")
    p_sources_dir = param.get("SOURCE_DIR")
    p_files = param.get("files")

    if p_chunk_size == CHUNK_SIZE and p_chunk_overlap == CHUNK_OVERLAP and p_sources_dir == SOURCES_DIR:
        removed_files = list(set(p_files) - set(files))
        new_files = list(set(files) - set(p_files))
        if removed_files:
            print(f"Les fichiers suivants ne sont plus présents : {removed_files}")
            nb_cleaned = chunk_eraser(removed_files)
            print(f"{nb_cleaned} fichiers de chunks ont été supprimés (dans '{OUTPUT_DIR}'/).")
            param["files"] = [f for f in p_files if f not in removed_files]
            write_json(os.path.join(OUTPUT_DIR, "param.json"), param)
        if not new_files:
            print(f"Aucun nouveaux fichier n'a été détecté (dans '{SOURCES_DIR}/').")
        else:
            print(f"{len(new_files)} nouveaux fichiers ont été détectés : {new_files} (dans '{SOURCES_DIR}/').")
        return new_files, param["files"]

def chunk_eraser(diff):
    nb_cleaned = 0
    for file in diff:
        prefix = os.path.splitext(file)[0]
        for f in os.listdir(OUTPUT_DIR):
            if f.startswith(prefix) and f.endswith(".txt"):
                file_path = os.path.join(OUTPUT_DIR, f)
                if os.path.isfile(file_path):
                    os.remove(file_path)
                    nb_cleaned += 1
    return nb_cleaned

files = [f for f in os.listdir(SOURCES_DIR) if f.endswith(".txt")]
old_files = []
if os.path.isfile(os.path.join(OUTPUT_DIR, "param.json")):
    files, old_files = chunk_cleaner(files)
else:
    os.makedirs(OUTPUT_DIR)
chunk_files(files, old_files)







Aucun nouveaux fichier n'a été détecté (dans 'transformed_sources/').
0 fichiers ont été découpés en chunks (dans 'chunks/CS_1000_CO_200/').


# Création de la base de données vectorielle


In [50]:
import os
import uuid
from sentence_transformers import SentenceTransformer
import chromadb

CHUNKS_NAME = "CS_1000_CO_200"
CHUNKS_DIR = f"chunks/{CHUNKS_NAME}"  # Dossier contenant les chunks
EMBEDDING_MODEL = "all-MiniLM-L6-v2"
VECTOR_DB_DIR = f"vector_db/{EMBEDDING_MODEL.replace('-', '_')}"

# Init Chroma
client = chromadb.PersistentClient(path=VECTOR_DB_DIR)
collection = client.get_or_create_collection(CHUNKS_NAME)

# Embedding model
model = SentenceTransformer(EMBEDDING_MODEL)

# Lecture des chunks
def load_chunks():
    chunks = []
    metadatas = []
    ids = []
    for file in os.listdir(CHUNKS_DIR):
        if file.endswith(".txt"):
            path = os.path.join(CHUNKS_DIR, file)
            with open(path, "r", encoding="utf-8") as f:
                content = f.read()
                chunks.append(content)
                metadatas.append({"filename": file})
                ids.append(str(uuid.uuid4()))  # identifiant unique
    return chunks, metadatas, ids

# Génération + insertion
def embed_and_store():
    print("Lecture des chunks...")
    texts, metadatas, ids = load_chunks()

    print("Génération des embeddings...")
    embeddings = model.encode(texts, show_progress_bar=True).tolist()

    print("Insertion dans la base vectorielle...")
    collection.add(
        documents=texts,
        embeddings=embeddings,
        metadatas=metadatas,
        ids=ids
    )

    print(f"{len(texts)} chunks ajoutés à la base vectorielle dans '{VECTOR_DB_DIR}'.")

embed_and_store()


Lecture des chunks...
Génération des embeddings...


Batches: 100%|██████████| 16/16 [00:15<00:00,  1.02it/s]


Insertion dans la base vectorielle...
508 chunks ajoutés à la base vectorielle dans 'vector_db/all_MiniLM_L6_v2'.


### Affichage des collections existantes

In [5]:
import chromadb
EMBEDDING_MODEL = "all-MiniLM-L6-v2"

client = chromadb.PersistentClient(path=f"vector_db/{EMBEDDING_MODEL.replace('-', '_')}")
collections = client.list_collections()

# client.delete_collection("CS_800_CO_160")

for col in collections:
    print(f"Nom : {col.name}, ID : {col.id}")



Nom : CS_1000_CO_100, ID : 1f389081-73c5-4daf-b838-bf88aa24ab5f
Nom : CS_1000_CO_150, ID : 4ee75404-7ce2-49f3-be03-f455ef30b653
Nom : CS_800_CO_80, ID : 53fcc4cb-7fb3-4cc9-8b49-e0eea43e1188
Nom : CS_300_CO_60, ID : 9cae8747-44df-47d3-8054-851d6d71bad4
Nom : CS_300_CO_30, ID : bf0699be-251c-45f6-9656-0a7d9c38bac6
Nom : CS_800_CO_160, ID : c8e7ab35-faf7-4d49-bcab-a43898a80a8d
Nom : CS_500_CO_100, ID : d2e7a108-12b2-4a48-8f9f-e83c371e60f5
Nom : CS_1000_CO_200, ID : dd9a73c8-df80-4627-bd14-4d734ae5108f
Nom : CS_500_CO_50, ID : e21c2070-abd4-4b08-8f56-efa70d4e1a00


# Test du modèle

In [4]:
import chromadb
from sentence_transformers import SentenceTransformer

# Configuration
CHUNKS_NAME = "CS_1000_CO_200"
EMBEDDING_MODEL = "all-MiniLM-L6-v2"
VECTOR_DB_DIR = VECTOR_DB_DIR = f"vector_db/{EMBEDDING_MODEL.replace('-', '_')}"
TOP_K = 10  # nombre de résultats à retourner

# Initialisation du client et du modèle
client = chromadb.PersistentClient(path=VECTOR_DB_DIR)
collection = client.get_collection(CHUNKS_NAME)
model = SentenceTransformer(EMBEDDING_MODEL)

def ask_question():
    # question = input("Pose ta question : ").strip()
    # if not question:
    #     print("Tu dois poser une question.")
    #     return
    question = "Je souhaite faire une attaque à l'épée sur un ennemi, comment dois-je procéder ?"

    print("Recherche des passages les plus pertinents...\n")

    # Embedding de la question
    query_embedding = model.encode([question])[0].tolist()

    # Recherche vectorielle
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=TOP_K,
        include=["documents", "metadatas", "distances"]
    )

    # Filtrer les résultats pour exclure ceux avec une distance >= 1
    filtered_docs = []
    filtered_metas = []
    filtered_dists = []
    for doc, meta, dist in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
        if dist < 1:
            filtered_docs.append(doc)
            filtered_metas.append(meta)
            filtered_dists.append(dist)



    # Affichage des résultats filtrés
    for i, (doc, meta, dist) in enumerate(zip(filtered_docs, filtered_metas, filtered_dists)):
        print(f"\n--- Résultat {i+1} ({meta['filename']}) - Similarité : {round((1 - dist) * 100, 2)}% ({round(dist, 3)}) ---")
        print(doc.strip()[:1000])

    filtered_results = {
        "documents": [filtered_docs],
        "metadatas": [filtered_metas],
        "distances": [filtered_dists]
    }
    return filtered_results, question
        
RESULTS, QUESTION = ask_question()


Recherche des passages les plus pertinents...


--- Résultat 1 (Warhammer 4 - Livre de base_chunk_268.txt) - Similarité : 0.48% (0.995) ---
à pleine 
vitesse vers l'ennemi le plus proche que vous pouvez voir afin 
d'attaquer. En général, la seule action que vous pouvez faire est 
un Test de Compétence d'Armes ou un Test d'Athlétisme pour 
atteindre un ennemi plus rapidement. De plus, vous pouvez faire 
un Test de Mêlée d'Action Gratuit à chaque Round pendant que 
vous lancez tout ce que vous avez dans vos attaques. Enfin, vous 
gagnez un bonus de +1 Bonus de Force, telle est votre férocité. 
Vous restez dans la Frénésie jusqu'à ce que tous les ennemis dans 
votre ligne de vue soient pacifiés, ou jusqu'à ce que vous receviez 
la condition Étourdie ou Inconsciente . Une fois votre Frénésie 
terminée, vous recevez immédiatement un état de Fatigue .  
Haine (Cible)
Vous êtes consumé par la Haine pour la Cible, qui est 
normalement un groupe de personnes ou de créatures, telles 
que'Hochlan

In [5]:
import requests

def get_llm_with_context_ollam(question, context, model="mistral"):
    context = "\n\n".join(RESULTS["documents"][0])
    prompt = f"""Tu es un assistant intelligent portant sur le jeu de role sur table Warhammer Fantasy.
    Ton role est de répondre aux questions concernant les règles du jeu ainsi que sur l'univers de Warhammer Fantasy.
    Tu dois répondre de manière concise et précise, en te basant sur les informations fournies dans le contexte ci-dessous.

    ### Contexte :

    {context}


    ### Question : 

    {question}
    

    ### Réponse :
    """

    response = requests.post(
        f"http://localhost:11434/api/generate",
        json={
            "model": model,
            "prompt": prompt,
            "stream": False,
        }
    )

    data = response.json()
    return data["response"]

question = QUESTION
context = "\n\n".join(RESULTS["documents"][0])
answer = get_llm_with_context_ollam(question, context)

print(f"\n--- Réponse de l'IA ---\n{answer.strip()}\n")


--- Réponse de l'IA ---
Pour effectuer une attaque à l'épée contre un adversaire, vous devrez tout d'abord vérifier si vous êtes dans la portée de l'épée et que le cible soit visible. Si cela est le cas, vous pouvez faire un test de compétence des armes pour effectuer votre attaque. En tant que personnage en frénésie, vous bénéficiez d'un bonus de +1 force.

   De plus, vous gagnez un bonus de +1 pour toutes vos attaques contre le groupe haïssé si vous avez la haine pour cette cible. Enfin, vous pouvez effectuer un test de melee d'action gratuit à chaque tour pour lancer une attaque supplémentaire sans action complète.

   Si vous avez la préférence ou les préjugés contre le groupe ennemis, vous subirez une pénalité de -10 à tous vos tests de sociabilité envers ce groupe.



In [11]:
import os

def extract_stream_from_pdf(pdf_path, obj_num, output_path):
    with open(pdf_path, 'rb') as f:
        data = f.read()

    start_marker = f"{obj_num} 0 obj".encode()
    stream_marker = b'stream\n'
    endstream_marker = b'endstream'

    start_obj = data.find(start_marker)
    if start_obj == -1:
        print("Objet non trouvé")
        return

    start_stream = data.find(stream_marker, start_obj) + len(stream_marker)
    end_stream = data.find(endstream_marker, start_stream)

    stream_data = data[start_stream:end_stream]

    with open(output_path, 'wb') as out:
        out.write(stream_data)

    print(f"Flux extrait dans {output_path}")

sources_path = os.path.join(os.getcwd(), "sources")
extract_stream_from_pdf(sources_path + '/output_uncompressed.pdf', 704, sources_path + '/image704.jp2')


Flux extrait dans c:\Projets\LLM assistant\sources/image704.jp2
